<a href="https://colab.research.google.com/github/aimat163/python/blob/main/%D0%9F%D0%BE%D0%BB%D1%83%D1%87%D0%B0%D0%B5%D0%BC_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5_%D0%B8%D0%B7_%D1%8D%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%BD%D0%BD%D0%BE%D0%B9_%D0%BA%D0%BE%D0%BC%D0%BC%D0%B5%D1%80%D1%86%D0%B8%D0%B8_%D1%8F%D0%BD%D0%B4%D0%B5%D0%BA%D1%81_%D0%B2_%D0%B3%D1%83%D0%B3%D0%BB_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tapi-yandex-metrika
import pandas as pd
import requests
from tapi_yandex_metrika import YandexMetrikaStats
from datetime import datetime, timedelta
import gspread

# Ставим дату, чтобы отчет выгружался за полную прошлую неделю



In [ ]:
def get_week_bounds_last():
    today = datetime.today()
    weekday = today.weekday()  # 0=понедельник
    start_of_this_week = today - timedelta(days=weekday)
    start_of_last_week = start_of_this_week - timedelta(days=7)
    end_of_last_week = start_of_this_week - timedelta(days=1)
    return start_of_last_week.date(), end_of_last_week.date()

In [ ]:
date_start, date_end = get_week_bounds_last()
date_start, date_end

(datetime.date(2025, 10, 27), datetime.date(2025, 11, 2))

In [ ]:
# Преобразуем в строки в формате "YYYY-MM-DD"
date_start_str = date_start.strftime("%Y-%m-%d")
date_end_str = date_end.strftime("%Y-%m-%d")

# Выгружаем данные из метрики

In [ ]:
#токен и счетчик

ACCESS_TOKEN = ""
COUNTER_ID =
client = YandexMetrikaStats(access_token=ACCESS_TOKEN)

In [ ]:
def get_ecommerce_data(date_start, date_end, metrics, dimensions, limit=100000):
    params = dict(
        ids=COUNTER_ID,
        date1=date_start,
        date2=date_end,
        metrics=metrics,
        dimensions=dimensions,
        filters="ym:s:isRobot=='No'AND ym:s:productCategoryLevel1 == 'Детские товары'",
        limit=limit,
        accuracy='full'
    )
    report = client.stats().get(params=params)
    columns = report.columns
    result = report().to_values()
    return pd.DataFrame(result, columns=columns)

In [ ]:
product_sales_data_changed = get_ecommerce_data(
    date_start=date_start_str,
    date_end=date_end_str,
    metrics="ym:s:users,ym:s:productBasketsUniq,ym:s:productPurchasedUniq,ym:s:usersPurchasePercentage",
    dimensions="ym:s:productCategoryLevel3"
)

In [ ]:
product_sales_data_changed.head(3)

,ym:s:productCategoryLevel3,ym:s:users,ym:s:productBasketsUniq,ym:s:productPurchasedUniq,ym:s:usersPurchasePercentage
0,Детские кровати,15721.0,1571.0,138.0,0.877807
1,Детские матрасы,9450.0,1301.0,306.0,3.238095
2,Интерактивные игрушки,5658.0,108.0,14.0,0.247437


# Отправляем данные в гугл-таблицу

In [ ]:
gc = gspread.service_account(filename='/content/adroit-citadel-332510-aac35322dada.json')

In [ ]:
spreadsheet = gc.open('Детство - Weekly')

In [ ]:
try:
    worksheet = spreadsheet.add_worksheet(title=f"{date_start_str} / {date_end_str}", rows='50', cols='5')
except gspread.exceptions.APIError:
    # Если лист уже есть, просто обращаемся к нему
    worksheet = spreadsheet.worksheet(f"{date_start_str} / {date_end_str}")

In [ ]:
data = [product_sales_data_changed.columns.values.tolist()] + product_sales_data_changed.values.tolist()

In [ ]:
worksheet.update('A1', data)

/tmp/ipython-input-3444643042.py:1: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data)


{'spreadsheetId': '1mteZyn2_jpKppCD5LDpX2QLQlrp94feliNNRGDIP0Ic',
 'updatedRange': "'2025-10-27 / 2025-11-02'!A1:E23",
 'updatedRows': 23,
 'updatedColumns': 5,
 'updatedCells': 115}